**Hidden Markov models for cracking codes**

In this homework will use an HMM work to solve some  substitution ciphers. 

You are given some short encrypted texts from various English and Russian authors. Each was encoded using a different character substitution cipher. You need to identify the author of each. Plaintext data is provided with examples of each of the authors.


This homework is worth **15 points** and is due by **2st Dec. 03:00**, please submit the results of the **TASK 5** (a list of files and names of the author/work) to Anytask in the following format: 

```
filename\tauthor\tfirst_3_lines_of_decoded_text
```
    
where 'filename' is the encrypted file from "encrypted/" and the name of the 'author' from "plaintext/" 

Using an HMM you will not be able to decode the text perfectly, but you'll see that it's often readable and you should be able to automatically identify the author by matching it against lines in the plaintext files.  




In [21]:
# Utilities for loading data from file and converting characters to integers and back.
import numpy as np
    
def get_char_to_int_mapping(path):
    # Load data from path and get mapping from characters to integers and back.
    characters = set()
    for line in open(path):
        characters.update(set([c for c in line.strip()]))
    char_to_int_mapping = dict([(char, i) for i, char in enumerate(sorted(list(characters)))])
    int_to_char_mapping = [char for char, i in sorted(char_to_int_mapping.items(), key=lambda x: x[1])]
    return char_to_int_mapping, int_to_char_mapping

def load_sequences(path, char_to_int_mapping):
    # Load data from path and map to integers using mapping.
    return [[char_to_int_mapping[c] for c in line.strip()] for line in open(path)]

def estimate_markov_model_from_sequences(sequences, num_states):
    # Estimate a Markov model based on the sequences (integers) provided.

    # pi[i] = Pr(s_0 = i)
    pi_counts = np.zeros(num_states)

    # A[i, j] = Pr(s_t = j | s_{t-1} = i)
    A_counts = np.zeros((num_states, num_states))

    for sequence in sequences:
        # Collect counts for initial state distribution (pi) and transition matrix (A)
        pi_counts[sequence[0]] += 1
        for i in range(1, len(sequence)):
            A_counts[sequence[i-1], sequence[i]] += 1
    
    # Normalize counts to obtain parameter estimates
    pi = pi_counts / np.sum(pi_counts)
    A = A_counts / np.sum(A_counts, axis=1, keepdims=True)

    return pi, A

**TASK 1**: Make the following block run by completing the method 'estimate_markov_model_from_sequences' above.

In [27]:
# Some data to use.
plaintext = 'plaintext/english.txt'
# plaintext = 'plaintext/shakespeare.txt'
# plaintext = 'plaintext/russian.txt'

ciphertext = './encrypted/encrypted-10.txt' # short sequences in english
# ciphertext = 'encrypted/encrypted-14.txt' # longer sequences in russian

# load a character to integer mapping and reverse                                                                                                         
char_to_int_mapping, int_to_char_mapping = get_char_to_int_mapping(plaintext)

# load sequences as ints                                                                                                                                  
plaintext_sequences = load_sequences(plaintext, char_to_int_mapping)
encrypted_sequences = load_sequences(ciphertext, char_to_int_mapping)

# estimate a markov model over characters                                                                                                                 
pi, A = estimate_markov_model_from_sequences(plaintext_sequences, len(char_to_int_mapping))

Below is a mostly implemented HMM.

In [28]:
class HMM():

    def __init__(self, observations_to_char_mapping={}, states_to_char_mapping={}):
        # Determine number of states and observation space. 
        self.num_states = len(states_to_char_mapping)
        self.num_outputs = len(observations_to_char_mapping)
        self.states_to_char_mapping = states_to_char_mapping
        self.observations_to_char_mapping = observations_to_char_mapping
       
        # Random initialization
        self.pi = np.random.rand(self.num_states)
        self.pi /= np.sum(self.pi)
        self.A = np.random.rand(self.num_states, self.num_states)
        self.A /= np.sum(self.A, 1, keepdims=True)
        self.B = np.random.rand(self.num_states, self.num_outputs)
        self.B /= np.sum(self.B, 1, keepdims=True) 
        
    def estimate_with_em(self, sequences, parameters={}, epsilon=0.001, max_iters=100):
        # Estimates all parameters not provided in 'parameters' based on 'sequences'.
        self.fixed_pi = 'pi' in parameters
        if self.fixed_pi:
            self.pi = parameters['pi']
        self.fixed_A = 'A' in parameters
        if self.fixed_A:
            self.A = parameters['A']
        self.fixed_B = 'B' in parameters
        if self.fixed_B:
            self.B = parameters['B']
    
        previous_llh = None
        for iter in range(max_iters):
            # Infer expected counts.
            pi_counts, A_counts, B_counts, log_likelihood = self.e_step(sequences)

            # Update parameters based on counts.
            self.m_step(pi_counts, A_counts, B_counts)

            # Output some sequences for debugging.
            self.output(sequences[:10])

            # Log likelihood should be increasing
            print('iteration %d; log likelihood %.4f' % (iter, log_likelihood))
            if previous_llh:
                assert log_likelihood >= previous_llh
                if log_likelihood - previous_llh < epsilon:
                    break
            previous_llh = log_likelihood


    def e_step(self, sequences):
        # Reset counters of statistics
        pi_counts = np.zeros_like(self.pi)
        A_counts = np.zeros_like(self.A) 
        B_counts = np.zeros_like(self.B) 
        total_log_likelihood = 0.0

        for sequence in sequences:
            # Run Forward-Backward dynamic program
            alpha, beta, gamma, xi, log_likelihood = self.forward_backward(sequence)
  
            # Accumulate statistics.
            pi_counts += gamma[:, 0]
            A_counts += xi
            for t, x in enumerate(sequence):
                B_counts[:, x] += gamma[:, t]
            
            total_log_likelihood += log_likelihood

        return pi_counts, A_counts, B_counts, total_log_likelihood

    def m_step(self, pi_counts, A_counts, B_counts):
        if not self.fixed_pi:
            self.pi = pi_counts / np.sum(pi_counts)
        if not self.fixed_A:
            self.A = A_counts / np.sum(A_counts, 1, keepdims=True)
        if not self.fixed_B:
            self.B = B_counts / np.sum(B_counts, 1, keepdims=True)
        
    def max_posterior_decode(self, sequence):
        _, _, gamma, _, log_likelihood = self.forward_backward(sequence)
        return np.argmax(gamma, 0)
        
    def forward_backward(self, sequence):
        # alpha[i][t] = p(x_1, ..., x_t, z_t = i)
        alpha = self.forward(sequence)
        
        # beta[i][t] = p(x_t+1, ..., x_T|z_t = i)
        beta = self.backward(sequence)

        # gamma[i][t] = p(z_t = i|x_1, ..., x_T)
        gamma = (alpha * beta) / np.sum(alpha * beta, 0)

        # xi[i][j] = p(z_t = i, z_{t+1} = j|x_1, ..., x_T)
        xi = np.zeros_like(self.A)
        for t in range(1, len(sequence)-1):
            this_xi = np.zeros_like(self.A)
            for i in range(self.num_states):
                for j in range(self.num_states):
                    this_xi[i, j] += alpha[i, t] * self.A[i, j] * beta[j, t+1] * self.B[j, sequence[t+1]]        
            xi += this_xi / np.sum(this_xi)
                
        return alpha, beta, gamma, xi, np.log(np.sum(alpha[:, len(sequence)-1]))
    
    def forward(self, obs):
        T = len(obs)
        alpha = np.zeros((self.A.shape[0], T))

        # base case
        alpha[:, 0] = self.pi * self.B[:, obs[0]]

        # recursive case
        for t in range(1, T):
            alpha[:, t] = np.dot(alpha[:, t-1], self.A) * self.B[:, obs[t]]

        return alpha

    def backward(self, obs):
        T = len(obs)
        beta = np.zeros((self.A.shape[0], T))

        # base case
        beta[:, T-1] = 1
        
        # recursive case
        for t in range(T-2, -1, -1):
            beta[:, t] = np.sum(beta[:, t+1] * self.A * self.B[:, obs[t+1]], 1)

        return beta

    def output(self, sequences):
        # Output some decoded states. 
        for i, sequence in enumerate(sequences):
            observations = [self.observations_to_char_mapping[x] for x in sequence]                
            map_states = [self.states_to_char_mapping[x] for x in self.max_posterior_decode(sequence)]
            print('(states):       %s\n(observations): %s' % (''.join(map_states), ''.join(observations)))


**TASK 2**: Implement the assertions in 'forward' and 'backward' methods on the HMM class so that the following block passes.

In [29]:
# Since it's a substitution cipher we assume hidden states and observations have same alphabet.
state_to_char_mapping = int_to_char_mapping
observation_to_char_mapping = int_to_char_mapping

# Initialize a HMM with the correct state/output spaces.
hmm = HMM(observation_to_char_mapping, state_to_char_mapping)

# Estimate the parameters and decode the encrypted sequences.
hmm.estimate_with_em(encrypted_sequences[:100], parameters={'pi': pi, 'A': A})

(states):       to    ote o  e o   e t  he    o e   o  o   oo  h
(observations): zcazdzcoqdcuwhdqzawhdxihqhgzhzcdgeuwczdbuzdqcetq
(states):       th    e  oe o   e te o      te      e      e 
(observations): tchihegdzchdqzaiqdogdqhfihzdogvjuhgfhdfemmhgz
(states):       thee t t    tee  o   n e  e t  e e te    oe
(observations): tchgdodxhifho hdzcazdmhgdaqdxjagzqdogfihaqh
(states):       to   e   e   o  ee   hee te  oe oe  o n  the
(observations): fchhihkdagkdfchfnhkdh hgdbsdzchdqhjvqamhdqns
(states):       w e  he  o te ee  oe e o e    o t i        oe
(observations): augzdogdzchoidseuzcvujdqaxdazdchowczdkhfihaqh
(states):       we     h  o te t  ee o          e n t  e
(observations): agkdthaidzchoidbia hdqzazhdeuzdevdmhmeis
(states):       ao e  oe   e  o   e  ote te   o  e  o  e
(observations): zchgdzchdfegfhozdevdzcoqdogfegqzagzdqzas
(states):       we e e   t e   t o te ee  o n      te to o 
(observations): qhzqdseudmeqzdiofcdogdseuzcdbhveihdmsdqowcz
(states):       th  e   o    e  

(states):       wene hen tend hith ie henth betene th wonon
(observations): qhzqdseudmeqzdiofcdogdseuzcdbhveihdmsdqowcz
(states):       the e windes t tone s bineth with set h
(observations): tchihdtaqzhvujdzomhdkhbazhzcdtozcdkhfas
(states):       te thinde hend s h at henth te n  ther noron
(observations): zedfcagwhdseuidkasdevdseuzcdzedqujjohkdgowcz
iteration 6; log likelihood -11594.3477
(states):       thit thin hare nline therenteth nondot but thech
(observations): zcazdzcoqdcuwhdqzawhdxihqhgzhzcdgeuwczdbuzdqcetq
(states):       whenean the nt oe in wethet int nente tentent
(observations): tchihegdzchdqzaiqdogdqhfihzdogvjuhgfhdfemmhgz
(states):       when o te teive thit men ar thinte inthe re
(observations): tchgdodxhifho hdzcazdmhgdaqdxjagzqdogfihaqh
(states):       the her ind thether aves by the we thine the
(observations): fchhihkdagkdfchfnhkdh hgdbsdzchdqhjvqamhdqns
(states):       annt in theid henthe t wit it heinot sethe re
(observations): augzdogdzchoidseuzcvujdqaxdazdch

KeyboardInterrupt: 

**TASK 3**: Some of the encrypted sequences are quite long. Try decoding some from 'encrypted/encrypted-5.txt' (note these are in Russian).

**TASK 4**: Make your implementation of forward and backward more efficient by removing all but the outermost for-loop.

**TASK 5**: Try to classify the author of each text by decoding some of it and matching it against the samples provided for each author.